In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile as shp 

In [3]:
map_results_folder = '../map_results_onshore' 

In [4]:
def get_elcc_list(region,year,capacity,technology,add_on=None,scale_down=False):
    elcc_map_file = '%s/%s_%d_%d_MW_%s%s_results_map.csv' % (map_results_folder,region,year,capacity,technology,'' if add_on is None else '_'+add_on)
    elcc = pd.read_csv(elcc_map_file,index_col=0,header=0).values
    elcc = elcc[1::2,1::2] if scale_down else elcc
    elcc = elcc.flatten()
    elcc = elcc[np.logical_not(np.isnan(elcc))]
    return elcc

In [5]:
def plot_box(ax, region, year, elcc):

    sns.set_theme(style="whitegrid")

    sns.boxplot(x=region, y=elcc, hue=year,ax=ax)
    ax.grid(False)
    ax.set_ylim(top=np.max(elcc)+5)
    return ax

In [6]:

fig, axs = plt.subplots(2,1,dpi=240, figsize=(7,8),constrained_layout=True)

for technology in ['solar','wind']:
    ax = axs[0 if technology=='solar' else 1]

    region_arr = np.array([])
    elcc = np.array([])
    year = np.array([])

    for region in ['basin','california','mountains','northwest','southwest']:
        
            elcc_2016 = get_elcc_list(region,2016,500,technology)
            elcc_2017 = get_elcc_list(region,2017,500,technology)
            elcc_2018 = get_elcc_list(region,2018,500,technology)
            elcc_2019 = get_elcc_list(region,2019,500,technology)
            
            year = np.append(year,['2016' for i in range(len(elcc_2016))]).flatten()
            year = np.append(year,['2017' for i in range(len(elcc_2017))]).flatten()
            year = np.append(year,['2018' for i in range(len(elcc_2018))]).flatten()
            year = np.append(year,['2019' for i in range(len(elcc_2019))]).flatten()

            elcc = np.append(elcc, np.array([elcc_2016,elcc_2017,elcc_2018,elcc_2019]).flatten())
            region_arr = np.append(region_arr, np.array([region.capitalize() for i in range(len(elcc_2016)*4)]))

    ax = plot_box(ax,region_arr, year, elcc)

    title = '%s Generators' % (technology.capitalize())
    ax.set_title(title)
    ax.set_ylabel('ELCC (%)')

img_name = '../gallery/SI_IAV'
plt.savefig(img_name)  
plt.close()